In [1]:
from ipyleaflet import *
from ipywidgets import *
from random import *

import pandas as pd
import numpy as np
import math

import xarray as xr
import os
import scipy

import pyqtree as qt
import bbox as bb

from collections import *
import operator
import time

In [2]:
class flightNode():

    
    def __init__(self, acid, latitude, longitude, time =None, altitude =None):
        
        self.acid = acid
        self.latitude = latitude
        self.longitude = longitude
        self.time = None
        self.altitude = None
        self.heat = None
        self.heatDistanceDelimiter = None

    
    def buildHeatBox(self, heatDistanceDelimiter):
        
        self.heatDistanceDelimiter = heatDistanceDelimiter
        xmin = ((self.getLatitude() * 6076) - heatDistanceDelimiter) / 6076
        xmax = ((self.getLatitude() * 6076) + heatDistanceDelimiter) / 6076
        ymin = ((self.getLongitude() * 6076 - heatDistanceDelimiter) / 6076) * math.cos(math.radians(xmin))
        ymax = ((self.getLongitude() * 6076 + heatDistanceDelimiter) / 6076) * math.cos(math.radians(xmax))
        self.heat = (xmin, ymin, xmax, ymax)

    def getAcid(self):
        
        return self.acid
    
    def getHeat(self):
        
        return self.heat
    
    def getLatitude(self):
        
        return self.latitude
    
    def getLongitude(self):
        
        return self.longitude
    
    def getAltitude(self):
        
        return altitude
    
    def getTime(self):
        
        return time
    
        

In [3]:
class flightNodeTrack():
    
    def __init__(self):
        
        self.flightNodes = []
        
    def addNode(self, node):
        
        self.flightNodes.append(node)
        
    def sendNodes(self):
        
        return self.flightNodes
        

In [4]:
class flightsOmni():
    
    def __init__(self):
        
        self.flightTracks = {}
        
    def addTrack(self, track):
        
        #nameOfDictionaryEntry = track.flightNodes[0].acid[:-1]
        self.flightTracks[track.flightNodes[0].acid[:-1]] = track
        
    def getFlights(self):
        
        return self.flightTracks

In [5]:
class mappDataFrame():
    
    def __init__(self, dataType = [], dataDate = None, airport = None):
    
        self.dataType = []
        self.dataDate = None
        self.airport = None
        self.adsbDate = None
        self.adsbDateHyphen = None
        self.tfms = None
        self.adsbDf = None
        self.tfmsDf = None
        self.aeroDf = None
        self.uasFlights = None
        self.adiHigh = None
        self.adiLow = None
        self.timeOut = []
        self.adsbFlightsDictionary = {}
        self.adsbFlights = None
        self.adsbLines = None
        self.adsbDisplay = None
        self.adsbHeat = {}
        self.uasOd = None
        self.uasLines = None
        self.uasDisplay = None
        self.uasHeat = {}
        self.heatDistanceDelimiter = None
        self.latitudeMax = None
        self.latitudeMin = None
        self.longitudeMax = None
        self.longitudeMin = None
        self.encounters = None
        
        
    def buildDataFrames(self, dataType, dataDate, airport = None):
        
        for dataTypes in dataType:
            self.dataType.append(dataTypes)
        
        if 'ADSB' in self.dataType:
            adsbDate = dataDate
            self.adsbDateHyphen = '-'.join(adsbDate)
            self.adsbDate = ''.join(adsbDate)
            self.adsbDf = pd.read_csv('./ADSB/' + self.adsbDate +'_csv.txt')
            
        
        if 'TFMS' in self.dataType:
            
            self.airport = airport
            self.tfmsDf = pd.read_csv('./TFMS/'+ self.airport + ".csv")
            
        if 'AERO' in self.dataType:
            
            xls = pd.ExcelFile('./AERO/aero8-14.xlsx')
            sheet_to_df_map = {}
            for sheet_name in xls.sheet_names:
                sheet_to_df_map[sheet_name] = xls.parse(sheet_name)
                sheet_to_df_map[sheet_name] = sheet_to_df_map[sheet_name].sort_values(['TimeWhen'])
                
            uasFlights = {}
            for df in sheet_to_df_map:

                uasFlights[str(sheet_to_df_map[df]['Drone ID'][0])] =[ [sheet_to_df_map[df]['Latitude'],
                                                                       sheet_to_df_map[df]['Longitude']],
                                                                      sheet_to_df_map[df]['TimeWhen'],
                                                                      sheet_to_df_map[df]['IconAltitude']]
            self.uasFlights = uasFlights

            
        
    def rebuildAdsbDf(self):
        
        if 'ADSB' in self.dataType:
            adsbDate = self.dataDate
            self.adsbDateHyphen = '-'.join(adsbDate)
            self.adsbDate = ''.join(adsbDate)
            self.adsbDf = pd.read_csv('./ADSB/' + self.adsbDate +'_csv.txt')

        
    def refreshParameters(self, adiLow, adiHigh, timeOut):
        
        self.adiLow = adiLow
        self.adiHigh = adiHigh
        self.timeOut = timeOut
        
    def refreshAltitude(self, adiLow, adiHigh):
        
        self.adiLow = adiLow
        self.adiHigh = adiHigh

    def refreshTime(self, timeOut):
        self.timeOut = timeOut
                    
    def refreshDate(self, date):
        self.dataDate = date
        
    def RGBtoHex(self, RGB):
        # Components need to be integers for hex to make sense
        RGB = [int(x) for x in RGB]
        return "#"+"".join(["0{0:x}".format(v) if v < 16 else
                "{0:x}".format(v) for v in RGB])


    def buildAdsbFlights(self):
        

        flights = []
        flights.extend({self.adsbDf.iat[0,11]})

        for y in range(len(self.adsbDf)):

            acidCur = self.adsbDf.iat[y - 1 , 11]

            if(self.adsbDf.iat[y, 11] != acidCur ):

                if(acidCur in flights):
                    ;
                else:
                    flights.extend([acidCur])

        for i in flights:

            if pd.isnull(i) == True:
                flights.remove(i)

        self.adsbFlights = flights
        
    def refreshAdsbFlights(self):

        adsbFlightsDictionary = {}
        adiLow = self.adiLow
        adiHigh = self.adiHigh
        timeOut = self.timeOut

        for items in self.adsbFlights:

            if type(items) == str:

                adsbFlightsDictionary.update({items+'track':[]})
                adsbFlightsDictionary.update({items+'time':[]})
                adsbFlightsDictionary.update({items+'altitude':[]})  
                #adsbFlightsDictionary.update({items+'heat': []})
                

        for x in range(len(self.adsbDf)):

            timeIn = self.adsbDf.iat[x,1]
            timeIn = time.strftime('%H:%M:%S', time.gmtime(timeIn))
            timeIn = ' '.join([self.adsbDateHyphen, timeIn])
            
            if type(self.adsbDf.iat[x,11]) == str:
                
                
                if timeIn >= self.adsbDateHyphen + timeOut[0] and timeIn <= self.adsbDateHyphen + timeOut[1]:
                    if int(self.adsbDf.iat[x,7]) >= adiLow and int(self.adsbDf.iat[x,7]) <= adiHigh:
                        adsbFlightsDictionary[self.adsbDf.iat[x,11] + "track"].append([self.adsbDf.iat[x,4] , self.adsbDf.iat[x,5]])    
                        adsbFlightsDictionary[self.adsbDf.iat[x,11] + "time"].append(timeIn)
                        adsbFlightsDictionary[self.adsbDf.iat[x,11] + "altitude"].append([self.adsbDf.iat[x,10], self.adsbDf.iat[x,7]])
                        #adsbFlightsDictionary[self.adsbDf.iat[x,11]].append(([self.adsbDf.iat[x,4] , self.adsbDf.iat[x,5]],
                                                                            #timeIn, self.adsbDf.iat[x,7]))
                        #adsbTree.insert()
                        
        self.adsbFlightsDictionary = adsbFlightsDictionary

    def refreshAdsbDisplay(self):
        
        lp = 0
        #colors = []
        lines = []
        colorRange = 255/len(self.adsbFlights)
        
        for items in self.adsbFlights:
            #colors.append('#%06X' % randint(0, 0xFFFFFF))
            #colorIn = colors[lp]
            #colorIn = 'blue'
            if lp == 0:
                colorIn = self.RGBtoHex([0,0,255])
            else:
                colorIn = self.RGBtoHex([ 0, int(255-(colorRange * lp)), 255 ])
            

            line = Polyline(
            locations = 
                [[
                    self.adsbFlightsDictionary[items + 'track']
                ,]],
                color = colorIn,
                fill_color = colorIn,
                fill = False,
                opacity = .7)

            if(type(self.adsbFlights[lp]) == str):
                line.popup=HTML(items)

            lines.append(line)
            lp+=1;
            
            
        self.adsbLines = lines
        if self.adsbDisplay == None:
            self.adsbDisplay = LayerGroup(layers=(lines)) 
        else:
            self.adsbDisplay.layers = self.adsbLines
            
            
    def refreshUasFlights(self):
        flightTracks = []
        flightTimes = []
        flightAltitudes = []
        uasOd = {}

        timeMin = self.adsbDateHyphen + self.timeOut[0]
        timeMax = self.adsbDateHyphen + self.timeOut[1]
        adiLow = self.adiLow
        adiHigh = self.adiHigh

        for z in self.uasFlights:
            tracks = []
            times = []
            altitudes = []
            heatIterator = 0
            for i in range(len(self.uasFlights[z][0][0])):
                k = self.uasFlights[z][0][0][i]
                j = self.uasFlights[z][0][1][i]
                time = self.uasFlights[z][1][i]
                altitude = self.uasFlights[z][2][i]
                if type(time) == str:
                    if time >= timeMin and time <= timeMax:
                         if altitude >= adiLow and altitude <= adiHigh:
                            #self.uasHeat.update({z:[]})
                            tracks.append([k,j])
                            times.append(time)
                            altitudes.append(altitude)
                            self.uasHeat.update({z + str(heatIterator):[]})
                            heatIterator+=1
                            
                            

            flightTimes.append(times)
            flightTracks.append(tracks)
            flightAltitudes.append(altitudes)
            uasOd[z] = (flightTimes, flightTracks,flightAltitudes)
            self.uasOd = uasOd

            
    def refreshUasDisplay(self):
        
        
        lp = 0
        #colors = []
        lines = []
        colorRange = 255/len(self.uasFlights)

        for i in self.uasFlights:
            #colors.append('#%06X' % randint(0, 0xFFFFFF))
            #colorIn = colors[lp]
            #colorIn = 'green'
            
            if lp == 0:
                colorIn = self.RGBtoHex([255,0,0])
            else:
                colorIn = self.RGBtoHex([ 255 , 0, int(255-(colorRange * lp)) ])

            
            line = Polyline(
            locations = 
                [[
                    self.uasOd[i][1][lp]
                ,]],
                color = colorIn,
                fill_color = colorIn,
                fill = False,
                opacity = .7)

            line.popup=HTML(i)
            lines.append(line)


            lp+=1;
            
        self.uasLines = lines
        if self.uasDisplay == None:
            self.uasDisplay = LayerGroup(layers=(lines)) 
        else:
            self.uasDisplay.layers = self.uasLines
             
    def refreshUasHeatTree(self, heatDistanceDelimiter):
        
        
        self.heatDistanceDelimiter = heatDistanceDelimiter
        
        iterator = 0
        uasTree =  qt.Index(bbox= (-90, -180, 90, 180))
        for uasFlight in self.uasFlights:
            uasTreeCoordIndex = 0
            for coordinate in self.uasOd[uasFlight][1][iterator]:
                
                xmin = ((coordinate[0] * 6076) - self.heatDistanceDelimiter) / 6076
                xmax = ((coordinate[0] * 6076) + self.heatDistanceDelimiter) / 6076
                ymin = ((coordinate[1] * 6076) - (self.heatDistanceDelimiter * math.cos(math.radians(xmin)))) / 6076
                ymax = ((coordinate[1] * 6076) + (self.heatDistanceDelimiter * math.cos(math.radians(xmax)))) / 6076
                self.uasHeat[uasFlight + str(uasTreeCoordIndex)] = (xmin, ymin, xmax, ymax)
                
                uasTree.insert(uasFlight + str(uasTreeCoordIndex), self.uasHeat[uasFlight + str(uasTreeCoordIndex)])
                uasTreeCoordIndex+=1
            iterator+=1
                
        self.uasTree = uasTree
        ;
        
        
    def refreshAdsbTree(self):
        

        adsbTree = qt.Index(bbox= (-90, -180, 90, 180))

        for flight in self.adsbFlights:
            flightTreeIndex = 0
            for coordinates in self.adsbFlightsDictionary[flight + 'track']:
                adsbTree.insert(flight + ',' + str(flightTreeIndex), (coordinates[0], coordinates[1], coordinates[0], coordinates[1]))
                flightTreeIndex+=1
        self.adsbTree = adsbTree
        
    def burnEncounterTrees(self):
        
        
        ;


In [6]:
mdf = mappDataFrame()

In [7]:
mdf.buildDataFrames(['ADSB', 'AERO', 'TFMS'], ['2019', '08', '24'], 'DAB')

In [8]:
mdf.refreshParameters(0,4000, [' 00:00:00', ' 24:00:00'])

In [9]:
mdf.buildAdsbFlights()
mdf.refreshAdsbFlights()
mdf.refreshAdsbDisplay()
mdf.refreshUasFlights()
mdf.refreshUasDisplay()

In [10]:
#mdf.adsbFlightsDictionary[mdf.adsbFlights[0]]

In [ ]:
zoomIn = 10
adiLow = 0
adiHigh = 12000
heatDistanceDelimiter = 101
heatHigh = 1010

In [12]:
mapp = Map(
    layers=(basemap_to_tiles(basemaps.Stamen.Toner), ),
    #center=[35.0433, -106.6129], zoom=zoomIn) # Albuquerque International Sunport : ABQ
    center = [29.1832, -81.0532], zoom=zoomIn) # Daytona Beach International Airport: DAB

mapp.add_layer(mdf.adsbDisplay)
mapp.add_layer(mdf.uasDisplay)
mapp.add_control(FullScreenControl()) 

In [13]:
mapp

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

**miscellaneous code to show functionality and time of execution**

In [18]:
mdf.refreshAdsbTree()

In [19]:
#mdf.uasHeat
mdf.refreshUasHeatTree(100)

In [20]:
results = []

for uasHeat in mdf.uasHeat:

    result = mdf.adsbTree.intersect(mdf.uasHeat[uasHeat])
    if len(result) > 0:
        results.append(result)
    

In [22]:
#results

take the results and find the center between them and the heat of the uas flight

In [ ]:
mdf.refreshDate(['2019', '09', '22'])